## Setup

In [1]:
from openai import OpenAI
import json
from dotenv import load_dotenv
import os

load_dotenv()

True

In [3]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
model = 'gpt-3.5-turbo-1106'

## Personal Function

In [4]:
def get_city_population(city):
    popuations = {
        "São Paulo": "12.33 million",
        "New York": "8.42 million",
        "Tokyo": "13.96 million"
    }
    return json.dumps({"city": city, "population": popuations.get(city, "unknown")})

## Firs API Call

In [5]:
messages = [{"role": "user", "content": "What is the population of São Paulo, New York and Tokyo?"}]

response = client.chat.completions.create(
    messages=messages,
    model=model,
    tools=[
        {
            "type": "function",
            "function":{
                "name": "get_city_population",
                "description": "Returns the population of a specific city",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "city": {
                            "type": "string",
                            "description": "The name of the city",
                        },
                    },
                    "required": ["city"]
                },
            },
        }
    ],
    tool_choice="auto"
)

In [6]:
print(response)

ChatCompletion(id='chatcmpl-9jTQxn9lCh0Vh85MVVg2WkjUMQZh2', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ng3ZLjh4Hx6rS0cM2ci1lV4n', function=Function(arguments='{"city": "São Paulo"}', name='get_city_population'), type='function'), ChatCompletionMessageToolCall(id='call_tsqKhElau1AEtQTQ2eoguqjv', function=Function(arguments='{"city": "New York"}', name='get_city_population'), type='function'), ChatCompletionMessageToolCall(id='call_OERRK2iC5JfRzNmEokpfyEn7', function=Function(arguments='{"city": "Tokyo"}', name='get_city_population'), type='function')]))], created=1720624931, model='gpt-3.5-turbo-1106', object='chat.completion', service_tier=None, system_fingerprint='fp_44132a4de3', usage=CompletionUsage(completion_tokens=64, prompt_tokens=68, total_tokens=132))


In [7]:
response_message = response.choices[0].message
print(response_message)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ng3ZLjh4Hx6rS0cM2ci1lV4n', function=Function(arguments='{"city": "São Paulo"}', name='get_city_population'), type='function'), ChatCompletionMessageToolCall(id='call_tsqKhElau1AEtQTQ2eoguqjv', function=Function(arguments='{"city": "New York"}', name='get_city_population'), type='function'), ChatCompletionMessageToolCall(id='call_OERRK2iC5JfRzNmEokpfyEn7', function=Function(arguments='{"city": "Tokyo"}', name='get_city_population'), type='function')])


In [8]:
tool_calls = response_message.tool_calls
print(tool_calls)

[ChatCompletionMessageToolCall(id='call_ng3ZLjh4Hx6rS0cM2ci1lV4n', function=Function(arguments='{"city": "São Paulo"}', name='get_city_population'), type='function'), ChatCompletionMessageToolCall(id='call_tsqKhElau1AEtQTQ2eoguqjv', function=Function(arguments='{"city": "New York"}', name='get_city_population'), type='function'), ChatCompletionMessageToolCall(id='call_OERRK2iC5JfRzNmEokpfyEn7', function=Function(arguments='{"city": "Tokyo"}', name='get_city_population'), type='function')]


## Funcion Call and Second API Call

In [9]:
if tool_calls:
      available_functions = {"get_city_population": get_city_population}
      messages.append(response_message)

      for tool_call in tool_calls:
          function_name = tool_call.function.name
          function_to_call = available_functions[function_name]
          function_args = json.loads(tool_call.function.arguments)
          function_response = function_to_call(city=function_args.get("city"))

          messages.append({
              "tool_call_id": tool_call.id,
              "role": "tool",
              "name": function_name,
              "content": function_response
          })

      new_response = client.chat.completions.create(
          model=model,
          messages=messages
      )

In [10]:
print(new_response)

ChatCompletion(id='chatcmpl-9jTwFcwrxtsnXLbX0ytnwPLqPUWpk', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The population of São Paulo is 12.33 million, New York is 8.42 million, and Tokyo is 13.96 million.', role='assistant', function_call=None, tool_calls=None))], created=1720626871, model='gpt-3.5-turbo-1106', object='chat.completion', service_tier=None, system_fingerprint='fp_44132a4de3', usage=CompletionUsage(completion_tokens=30, prompt_tokens=149, total_tokens=179))


In [12]:
from IPython.display import display, HTML

def display_wrapped_text(text):
    wrapped_text = "<p style='word-wrap: break-word; width: 800px;'>" + text + "</p>"
    display(HTML(wrapped_text))

long_text = new_response.choices[0].message.content
display_wrapped_text(long_text)